## 最終課題

In [12]:
import requests
from bs4 import BeautifulSoup
import time
# URLの結合や解析に使用
from urllib.parse import urljoin, urlparse
# 基本設定
base_url = 'https://www.musashino-u.ac.jp/'
domain = urlparse(base_url).netloc
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# 結果を格納する辞書 (Key: URL, Value: ページタイトル)
page_titles_dict = {}

# これから訪問するURLを格納するセット（キューの代わり）
urls_to_visit = set()

# 訪問済みURLを格納するセット
visited_urls = set()

# 除外するファイルの拡張子リスト
EXCLUDED_EXTENSIONS = [
    # 画像
    '.jpg', '.jpeg', '.png', '.gif', '.bmp', '.svg', '.webp', '.ico',
    # PDF
    '.pdf',
    # その他ドキュメント
    '.doc', '.docx', '.xls', '.xlsx', '.ppt', '.pptx',
    # 書庫
    '.zip', '.rar', '.lzh',
    # メディア
    '.mp3', '.mp4', '.mov', '.avi', '.wmv'
]

# 最初にトップページを訪問先に追加
urls_to_visit.add(base_url)
print(f"対象ドメイン: {domain}")

# 訪問予定リストが空になるまでループ
while urls_to_visit:
    
    # 訪問予定リストからURLを1つ取り出す
    current_url = urls_to_visit.pop()
    # 既訪問リストに追加
    visited_urls.add(current_url)
    # 処理件数をカウント
    page_count = len(visited_urls)

    try:
        # サーバーへの負荷軽減のための待機時間
        time.sleep(0.05) 
        res = requests.get(current_url, headers=headers, timeout=10)
        # ステータスコード 200 (OK) の場合のみ処理
        if res.status_code == 200:
            res.encoding = res.apparent_encoding
            # HTML以外の場合はパースしない
            if 'text/html' not in res.headers.get('Content-Type', ''):
                page_titles_dict[current_url] = 'HTML以外のコンテンツ'
                print(f"[{page_count}] {current_url} - スキップ (HTMLではありません)")
                continue # 次のループ
            soup = BeautifulSoup(res.text, 'html.parser')
            # <title>タグの検索と辞書への格納
            if soup.title and soup.title.string:
                title = soup.title.string.strip()
                page_titles_dict[current_url] = title
                print(f"[{page_count}] {current_url} - {title}")
            else:
                page_titles_dict[current_url] = 'タイトルなし'
                print(f"[{page_count}] {current_url} - タイトルなし")

            # ページ内のすべてのリンクを抽出
            links = soup.find_all('a', href=True)
            new_links_found = 0
            for link in links:
                href = link.get('href').strip()
                # mailto, tel, javascriptリンクを除外
                if href.startswith(('mailto:', 'tel:', 'javascript:')):
                    continue
                # ページ内リンクを除外
                if href.startswith('#'):
                    continue
                # 絶対URLを生成
                full_url = urljoin(base_url, href)
                # URLの末尾のフラグメントを削除
                full_url = full_url.split('#')[0]
                # 拡張子をチェックし、除外対象か判定
                path = urlparse(full_url).path
                is_excluded = any(path.lower().endswith(ext) for ext in EXCLUDED_EXTENSIONS)
                # ドメインが同じで、除外対象でなく、未訪問の場合のみ追加
                if (urlparse(full_url).netloc == domain and 
                    not is_excluded and 
                    full_url not in visited_urls and
                    full_url not in urls_to_visit):
                    urls_to_visit.add(full_url)
                    new_links_found += 1
        else:
            # ステータスコードが200以外の場合の処理
            page_titles_dict[current_url] = f'アクセスエラー (Code: {res.status_code})'
            print(f"[{page_count}] {current_url} - スキップ (ステータスコード: {res.status_code})")


    except requests.exceptions.Timeout:
        page_titles_dict[current_url] = 'アクセスエラー (タイムアウト)'
        print(f"[{page_count}] {current_url} - タイムアウトエラー")
    except requests.exceptions.SSLError:
        page_titles_dict[current_url] = 'アクセスエラー (SSLエラー)'
        print(f"[{page_count}] {current_url} - SSLエラー")
    except requests.exceptions.RequestException as e:
        page_titles_dict[current_url] = f'アクセスエラー ({type(e).__name__})'
        print(f"[{page_count}] {current_url} - アクセスエラー ({type(e).__name__})")  
    except Exception as e:
        # その他の予期せぬエラー
        page_titles_dict[current_url] = f'不明なエラー ({type(e).__name__})'
        print(f"[{page_count}] {current_url} - 不明なエラー ({e})")

# ループが正常に終了
print("\n--------------------------------------------------")
print(f"クロール処理が完了しました ({page_count} 件のページを処理)")
print(f"収集した総ページ数: {len(page_titles_dict)} 件")

対象ドメイン: www.musashino-u.ac.jp
[1] https://www.musashino-u.ac.jp/ - 武蔵野大学
[2] https://www.musashino-u.ac.jp/academics/teachers_license/ - 教職課程・国家資格 | 学部・大学院 | 武蔵野大学
[3] https://www.musashino-u.ac.jp/guide/profile/message.html - 学長就任のごあいさつ | 大学案内 | 武蔵野大学
[4] https://www.musashino-u.ac.jp/admission/request.html - 資料請求 | 入試情報 | 武蔵野大学
[5] https://www.musashino-u.ac.jp/happiness_creators/no026.html - 国際協力で国をつなぐ架け橋に―カレン族の村での活動と大使館での経験 | 武蔵野大学
[6] https://www.musashino-u.ac.jp/harassment/ - ハラスメント防止・対策 | 武蔵野大学
[7] https://www.musashino-u.ac.jp/research/alignment/collaborativeresearch.html - 連携研究の取り組み | 研究 | 武蔵野大学
[8] https://www.musashino-u.ac.jp/guide/information/degree.html - 取得可能学位 | 大学案内 | 武蔵野大学
[9] https://www.musashino-u.ac.jp/research/laboratory/cognitive_behavioral_therapy.html - 認知行動療法研究所 | 研究 | 武蔵野大学
[10] https://www.musashino-u.ac.jp/guide/campus/ariake_campus.html - 有明キャンパス | 大学案内 | 武蔵野大学
[11] https://www.musashino-u.ac.jp/news/detail/20251030-7383.html - 本学公認クラブ「和太鼓 隼」と「武蔵野大学ウインドア